Import dependencies

In [1]:
import os
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor
from tqdm import tqdm

**1. Load environment**

In [2]:
environment_name = 'CartPole-v1'
env = gym.make(environment_name, render_mode='rgb_array')
env = Monitor(env)

# action space:         env.action_space
# obersevation space:   env.observation_space
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action = env.action_space.sample() # take random action
        n_state, reward, done, trunc, info = env.step(action) # apply action to environment
        score += reward
    print(f'Episode: {episode}, Score: {score}')
env.close()

Episode: 1, Score: 14.0
Episode: 2, Score: 13.0
Episode: 3, Score: 39.0
Episode: 4, Score: 16.0
Episode: 5, Score: 11.0


Understand the environment

In [3]:
env.action_space

Discrete(2)

In [4]:
env.action_space.sample()

0

In [5]:
env.observation_space

Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)

In [6]:
env.observation_space.sample()

array([ 1.0088098e-01, -2.2695049e+38,  3.9206123e-01, -6.4406351e+37],
      dtype=float32)

**2. Training**

In [7]:
log_path = os.path.join('Training', 'Logs')
# Create the directory if it doesn't exist
if not os.path.exists(log_path):
    os.makedirs(log_path)

In [16]:
env = gym.make(environment_name, render_mode='rgb_array')
env = Monitor(env)
env = DummyVecEnv([lambda: env]) # The algorithms require a vectorized environment to run with lambda as a function
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path) # MlpPolicy is a simple policy that only has one hidden layer with 64 neurons
# Stable baselines 3 has three policy types: MlpPolicy, CnnPolicy, MultiInputPolicy

Using cpu device


In [10]:
#PPO??
model.learn(total_timesteps=20000)

Logging to Training/Logs/PPO_5
-----------------------------
| time/              |      |
|    fps             | 5392 |
|    iterations      | 1    |
|    time_elapsed    | 0    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 3302        |
|    iterations           | 2           |
|    time_elapsed         | 1           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008760935 |
|    clip_fraction        | 0.0804      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.687      |
|    explained_variance   | 0.0113      |
|    learning_rate        | 0.0003      |
|    loss                 | 8.39        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0127     |
|    value_loss           | 56.4        |
-----------------------------------------
---

**3. Save and reload model**

In [11]:
PPO_path = os.path.join('Training', 'Saved Models', 'PPO_Model_Cartpole')

In [12]:
model.save(PPO_path)

In [13]:
del model

In [20]:
model = PPO.load(PPO_path, env=env)

**4. Evaluate model**

In [29]:
# PPO model is considered solved for score 500
evaluate_policy(model, env, n_eval_episodes=10, render=True)

(500.0, 0.0)

In [22]:
env.close()

**5. Test model**

In [30]:
episodes = 5
for episode in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action, _ = model.predict(obs) # take action from model
        obs, reward, done, info = env.step(action) # apply action to environment
        score += reward
    print(f'Episode: {episode}, Score: {score}')
#env.close()

Episode: 1, Score: [300.]
Episode: 2, Score: [331.]
Episode: 3, Score: [500.]
Episode: 4, Score: [500.]
Episode: 5, Score: [500.]


In [34]:
obs = env.reset()

In [35]:
action, _ = model.predict(obs)

In [36]:
env.step(action) # reward is 1 for every step

(array([[ 0.01614755,  0.23278338,  0.00951417, -0.33587608]],
       dtype=float32),
 array([1.], dtype=float32),
 array([False]),
 [{'TimeLimit.truncated': False}])

**6. View logs in Tensorboard**

In [40]:
training_log_path = os.path.join(log_path, 'PPO_2')

In [45]:
!tensorboard --logdir={training_log_path}

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.16.2 at http://localhost:6006/ (Press CTRL+C to quit)
^C


In [ ]:
# core metrics to look at:
# 1. average reward
# 2. avrage episode length

# training strategies:
# 1. train for longer
# 2. hyperparameter tuning

**7. Adding a callback to the training stage**

In [46]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

In [47]:
save_path = os.path.join('Training', 'Saved Models')

In [48]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=500, verbose=1)
eval_callback = EvalCallback(env, callback_on_new_best=stop_callback, eval_freq=10000, best_model_save_path=save_path, verbose=1)

In [49]:
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cpu device


In [50]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to Training/Logs/PPO_6
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 21.6     |
|    ep_rew_mean     | 21.6     |
| time/              |          |
|    fps             | 5489     |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 26.9        |
|    ep_rew_mean          | 26.9        |
| time/                   |             |
|    fps                  | 3536        |
|    iterations           | 2           |
|    time_elapsed         | 1           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.009472036 |
|    clip_fraction        | 0.113       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | 0.00179     |

**8. Change policy**

In [56]:
net_arch = dict(pi=[128,128,128,128], vf=[128,128,128,128])

In [57]:
model = PPO('MlpPolicy', env, verbose=1, policy_kwargs={'net_arch':net_arch}, tensorboard_log=log_path)

Using cpu device


In [60]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to Training/Logs/PPO_9
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 456      |
|    ep_rew_mean     | 456      |
| time/              |          |
|    fps             | 4182     |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 2048     |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 478          |
|    ep_rew_mean          | 478          |
| time/                   |              |
|    fps                  | 498          |
|    iterations           | 2            |
|    time_elapsed         | 8            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0050710468 |
|    clip_fraction        | 0.0916       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.43        |
|    explained_variance   |

**9. Using an alternate algorithm**

In [61]:
from stable_baselines3 import DQN

In [62]:
model = DQN('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cpu device


In [ ]:
model.learn(total_timesteps=20000)